- [Исследование надежности заемщиков.](#toc1_)    
    - [Шаг 1. Обзор данных](#toc1_1_1_)    
    - [Шаг 2.1 Заполнение пропусков](#toc1_1_2_)    
    - [Шаг 2.2 Проверка данных на аномалии и исправления.](#toc1_1_3_)    
    - [Шаг 2.3. Изменение типов данных.](#toc1_1_4_)    
    - [Шаг 2.4. Удаление дубликатов.](#toc1_1_5_)    
    - [Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.](#toc1_1_6_)    
    - [Шаг 2.6. Категоризация дохода.](#toc1_1_7_)    
    - [Шаг 2.7. Категоризация целей кредита.](#toc1_1_8_)    
    - [Ответы на вопросы.](#toc1_1_9_)    
        - [Вопрос 1:](#toc1_1_9_1_1_)    
        - [Вывод 1:](#toc1_1_9_1_2_)    
        - [Вопрос 2:](#toc1_1_9_1_3_)    
        - [Вывод 2:](#toc1_1_9_1_4_)    
        - [Вопрос 3:](#toc1_1_9_1_5_)    
        - [Вывод 3:](#toc1_1_9_1_6_)    
        - [Вопрос 4:](#toc1_1_9_1_7_)    
        - [Вывод 4:](#toc1_1_9_1_8_)    
  - [Общий вывод:](#toc1_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Исследование надежности заемщиков.](#toc0_)

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку

### <a id='toc1_1_1_'></a>[Шаг 1. Обзор данных](#toc0_)

In [35]:
import pandas as pd
data = pd.read_csv("/datasets/data.csv")
data.info()
data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4651,0,375129.070606,71,среднее,1,вдовец / вдова,2,F,пенсионер,0,138706.135238,покупка жилья
20650,0,344746.326618,65,среднее,1,женат / замужем,0,F,пенсионер,0,71425.048805,недвижимость
9679,2,-7920.458750,40,среднее,1,гражданский брак,1,F,госслужащий,0,209465.965530,сыграть свадьбу
6520,0,-2271.003374,58,среднее,1,женат / замужем,0,M,сотрудник,0,331863.267393,операции с недвижимостью
916,1,-193.894643,39,начальное,3,женат / замужем,0,M,сотрудник,0,165491.893316,заняться образованием
15014,1,335066.543203,45,среднее,1,в разводе,3,M,пенсионер,0,206999.918673,образование
2213,0,-976.884186,32,среднее,1,женат / замужем,0,F,сотрудник,0,207597.541367,ремонт жилью
8746,0,-620.992337,58,среднее,1,гражданский брак,1,M,сотрудник,0,225933.614926,свадьба
14142,0,-7435.604522,53,высшее,0,гражданский брак,1,M,компаньон,0,192393.078743,заняться высшим образованием
17667,0,-4985.734895,45,среднее,1,гражданский брак,1,F,компаньон,0,191986.562579,на покупку автомобиля


### <a id='toc1_1_2_'></a>[Шаг 2.1 Заполнение пропусков](#toc0_)

В ходе изучения общей информации о датасете установлено, что колонках days_employed и total_income имеются пустые значения. 

In [36]:
if list(data['days_employed'].isnull()) == list(data['total_income'].isnull()): 
    print('совпадают')
else: 
    print('не совпадают')

совпадают


При пропуске значений в столбце days_employed имеется и пропуск в столбце total_income,таким образом значения в рассматриваемых 
столбцах пропущены одинаковое число раз и в одних и тех же строках (это наблюдение будет использовано при ответе на вопросы 
                                                                    подпункта). 

In [37]:
print('Кол-во пропусков в столбцах: days_employed, total_income  равняется: {0}, \n\
что состоявлет {1:.1f}% от общего числа стоблцов'.
      format(data['days_employed'].isnull().sum(), 
             data['days_employed'].isnull().sum() * 100 / data['days_employed'].count()))
print(data['total_income'].isnull().sum(), data['total_income'].isnull().sum() * 100 / data['total_income'].count()) # перепроверка по столцу total_income

Кол-во пропусков в столбцах: days_employed, total_income  равняется: 2174, 
что состоявлет 11.2% от общего числа стоблцов
2174 11.234561521368406


In [38]:
print(data[data['total_income'].isnull()]['income_type'].value_counts()) # узнаем тип занятости лиц в строках с пропущенными значениями

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Это информация потребуется для выводов

In [39]:
med = data['total_income'].median() # определение медианного значения
data['total_income'] = data['total_income'].fillna(med)
data.info() # проверка

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Ответы на вопросы подпункта 1 Шага 2 (далее - пп.). 
- обнаружены пропущенные значения в столбцах days_employed и total_income, 
то есть общий трудовой стаж в днях и ежемесячный доход соответственно; 
- кол-во пропусков в столбцах: days_employed, total_income  равняется: 2174,
что составляет 11.2% от общего числа столбцов; 
- прежде чем делать выводы о причинах пропуска значений в столбцах days_employed, total_income
необходимо отметить следующее:
    1. Пропуски в столбцах days_employed, total_income возникают в одних и тех же строках.
    2. Значения в вышеуказанных столбцах пропущены в отношении 413 пенсионеров и 147 госслужащих.
    3. В рассматриваемом датасете множество лиц с пропуском значения days_employed 
полностью совпадает с множеством лиц с пропуском значения total_income, что логично (так как при отсутствии 
трудового стажа, как правило нет дохода), что говорит, о том, что это не техническая ошибка и 
позволяет сделать следующий вывод: вышеуказанные лица либо не работали либо работают неофициально и 
не могли при оформлении кредита предоставить документы, подтверждающие наличие и размер трудового стажа, дохода.
    Однако среди рассматриваемой категории граждан имеется 413 пенсионеров и 147 госслужащих, 
которые не могут не иметь доход, а также документы его подтверждающие.
    Если бы я работал с анализируемым датасетом на реальной работе, то я бы обратился к лицу, предоставившему данные за разъяснением.
    
С учетом описания задания к датасету я делаю вывод, что пропуски в столбах возникли в следствие технической ошибки.
 -  Заполнить пропуски медианным значением — лучшее решение для количественных переменных, 
    потому что на значение среднего могут существенно влиять крайние значения.  

### <a id='toc1_1_3_'></a>[Шаг 2.2 Проверка данных на аномалии и исправления.](#toc0_)

In [40]:
print("Количество отрицательных строк в days_employed:", 
      data[data['days_employed'] < 0]['days_employed'].count())
data['days_employed'] = abs(data['days_employed']) # Меняем отрицательные значения на положительные
print("Количество отрицательных строк в days_employed:", 
      data[data['days_employed'] < 0]['days_employed'].count()) # проверка


Количество отрицательных строк в days_employed: 15906
Количество отрицательных строк в days_employed: 0


In [41]:
columns_cheсk = ['children', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id',
                 'gender', 'income_type', 'debt', 'purpose']

for name_columns in columns_cheсk: 
    print(name_columns, sorted(data[name_columns].unique())) # выводим уникальные значения для столбцов для поиска аномалий

children [-1, 0, 1, 2, 3, 4, 5, 20]
dob_years [0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
education ['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее', 'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']
education_id [0, 1, 2, 3, 4]
family_status ['Не женат / не замужем', 'в разводе', 'вдовец / вдова', 'гражданский брак', 'женат / замужем']
family_status_id [0, 1, 2, 3, 4]
gender ['F', 'M', 'XNA']
income_type ['безработный', 'в декрете', 'госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник', 'студент']
debt [0, 1]
purpose ['автомобили', 'автомобиль', 'высшее образование', 'дополнительное образование', 'жилье', 'заняться высшим образованием', 'заняться образованием

В колонке children имеются значения -1 и 20, что очевидно аномалия. 

In [45]:
data['children'] = abs(data['children'])
data.loc[data['children'] == 20, 'children'] = 2
print('children', sorted(data['children'].unique())) # проверка

children [0, 1, 2, 3, 4, 5]


Возможными причинами появления таких данных является техническая ошибка, человеческий фактор.

### <a id='toc1_1_4_'></a>[Шаг 2.3. Изменение типов данных.](#toc0_)

In [46]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Осуществлена замена в столбце total_income на целочисленное значение

### <a id='toc1_1_5_'></a>[Шаг 2.4. Удаление дубликатов.](#toc0_)

In [47]:
print('Число дубликатов в датасете:', data.duplicated().sum())
data = data.drop_duplicates().drop_duplicates().reset_index(drop=True)
print('Число дубликатов в датасете:', data.duplicated().sum()) # Проверка

Число дубликатов в датасете: 54
Число дубликатов в датасете: 0


'Поиск не явных дубликатов:'

In [48]:
# Здесь дублирую предыдущий код для удобства восприятия
columns_cheсk = ['children', 'dob_years', 'education', 'family_status', 'gender', 'income_type', 'debt', 'purpose']
for name_columns in columns_cheсk: 
    print(name_columns, sorted(data[name_columns].unique()))

children [0, 1, 2, 3, 4, 5]
dob_years [0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
education ['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее', 'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']
family_status ['Не женат / не замужем', 'в разводе', 'вдовец / вдова', 'гражданский брак', 'женат / замужем']
gender ['F', 'M', 'XNA']
income_type ['безработный', 'в декрете', 'госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник', 'студент']
debt [0, 1]
purpose ['автомобили', 'автомобиль', 'высшее образование', 'дополнительное образование', 'жилье', 'заняться высшим образованием', 'заняться образованием', 'на покупку автомобиля', 'на покупку подержанного автомобиля', 'на 

В колонке education неявные дубликаты выражены в наличии записей одного и того же уровня образования в разных регистрах.

In [49]:
data['education'] = data['education'].str.lower() # Приведение к нижнему регистру
print(sorted(data['education'].unique())) # проверка
print('Число дубликатов в датасете:', data.duplicated().sum())
data = data.drop_duplicates().drop_duplicates().reset_index(drop=True)
print('Число дубликатов в датасете:', data.duplicated().sum()) # Проверка

['высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']
Число дубликатов в датасете: 17
Число дубликатов в датасете: 0


Ответы на вопросы пп.4
- для поиска и удаление дубликатов использовался метод duplicated(), 
для поиска не явных дубликатов использовался метод unique(), так как 
он выводит только уникальные значения; 
- возможные причины появление дубликатов ошибка при выгрузке данных, 
человеческий фактор

### <a id='toc1_1_6_'></a>[Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.](#toc0_)

In [50]:
education_dict = data[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
print('education_dict', education_dict)
family_dict = data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
print('family_status', family_dict)
data = data.drop(['education', 'family_status'], axis=1)
data.info()

education_dict              education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4
family_status            family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education_id      21454 non-null  int64  
 4   family_status_id  21454 non-null  int64  
 5   gender            21454 non-null  object 
 6   i

### <a id='toc1_1_7_'></a>[Шаг 2.6. Категоризация дохода.](#toc0_)

In [51]:
def income_group(amount):
    if 0 <= amount <= 30000: 
        return "E"
    elif 30001 <= amount <= 50000: 
        return "D"
    elif 50001 <= amount <= 200000: 
        return "C"
    elif 200001 <= amount <= 1000000: 
        return "B"
    elif 1000001 <= amount: 
        return "A"
    
data['total_income_category'] = data['total_income'].apply(income_group)
print(data[['total_income', 'total_income_category' ]].sample(10)) # проверка

       total_income total_income_category
17320        109759                     C
20084        145017                     C
20350        145017                     C
15983        145017                     C
3982         232010                     B
18418        192454                     C
20905        224281                     B
7456         122553                     C
2591          38275                     D
1793         172723                     C


### <a id='toc1_1_8_'></a>[Шаг 2.7. Категоризация целей кредита.](#toc0_)

Для выполнения шага 2.7 создаем функцию (processing_value_colums), принимающую аргумент - строку (name_purpose). 
Функция содержит в себе словарь(category_name), ключами которого являются категории кредита, значением словаря являются ключевые слова, использованные в датасте для описания целей кредита. 
Функция возвращает строку с названием соответствующей категории при совпадении целей кредита с ключевыми словами. 

In [52]:
def processing_value_colums(name_purpose):
    dict_category_name = {'операции с автомобилем': ['авто'], 
              'операции с недвижимостью': ['недвиж', 'жил'],
              'получение образования': ['образ'],
              'проведение свадьбы': ['свадьб']}
    for category_name in dict_category_name.keys(): # получаем значение 
        for keyword in dict_category_name[category_name]: # проходим по ключевым словам
                if keyword in name_purpose: # ищем соответствие 
                    return category_name
data['purpose_category'] = data['purpose'].apply(processing_value_colums)
data.info()
print(data['purpose_category'].unique())
data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21454 non-null  int64  
 1   days_employed          19351 non-null  float64
 2   dob_years              21454 non-null  int64  
 3   education_id           21454 non-null  int64  
 4   family_status_id       21454 non-null  int64  
 5   gender                 21454 non-null  object 
 6   income_type            21454 non-null  object 
 7   debt                   21454 non-null  int64  
 8   total_income           21454 non-null  int64  
 9   purpose                21454 non-null  object 
 10  total_income_category  21454 non-null  object 
 11  purpose_category       21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
['операции с недвижимостью' 'операции с автомобилем'
 'получение образования' 'проведение свадьбы']


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
3738,0,2083.882774,37,1,4,M,сотрудник,0,78495,свой автомобиль,C,операции с автомобилем
11376,0,842.501580,57,0,4,M,компаньон,0,120554,автомобили,C,операции с автомобилем
10188,0,681.569580,34,1,0,M,сотрудник,0,272971,операции со своей недвижимостью,B,операции с недвижимостью
6781,0,5582.828856,51,1,0,M,сотрудник,0,122973,дополнительное образование,C,получение образования
16384,0,8546.554301,43,1,0,M,сотрудник,0,100817,строительство жилой недвижимости,C,операции с недвижимостью
18969,0,347722.358671,63,1,0,F,пенсионер,0,279112,жилье,B,операции с недвижимостью
21426,0,NaN,48,0,0,F,компаньон,0,145017,строительство недвижимости,C,операции с недвижимостью
9502,0,3906.862845,55,1,1,F,сотрудник,0,191502,свадьба,C,проведение свадьбы
12351,1,1258.754340,30,1,4,M,сотрудник,0,183204,ремонт жилью,C,операции с недвижимостью
2375,0,55.475648,44,1,0,F,компаньон,0,168682,на покупку своего автомобиля,C,операции с автомобилем


Пустых значений в столбце purpose_category нет, значит функция успешно обработала все необходимые значения

### <a id='toc1_1_9_'></a>[Ответы на вопросы.](#toc0_)

##### <a id='toc1_1_9_1_1_'></a>[Вопрос 1:](#toc0_)

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [53]:
children_group = data.groupby(['children']).agg({'children':'count', 'debt':'sum'})
children_group['persent_debt'] = children_group['debt'] * 100 / children_group['children']
print(children_group)
print('Процент просрочек по кредитам клиентов с детьми:', children_group[1:]['debt'].sum() * 100 / 
      children_group.loc[1:]['children'].sum())

          children  debt  persent_debt
children                              
0            14091  1063      7.543822
1             4855   445      9.165808
2             2128   202      9.492481
3              330    27      8.181818
4               41     4      9.756098
5                9     0      0.000000
Процент просрочек по кредитам клиентов с детьми: 9.208203178052424


##### <a id='toc1_1_9_1_2_'></a>[Вывод 1:](#toc0_)

У людей с детьми чаще просрочки по кредитам (9.2%), чем у бездетных (7.5%)

##### <a id='toc1_1_9_1_3_'></a>[Вопрос 2:](#toc0_)

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [54]:
family_group = data.groupby(['family_status_id']).agg({'family_status_id':'count', 'debt':'sum'})
family_group['persent_debt'] = family_group['debt'] * 100 / family_group['family_status_id']
print(family_dict)
print(family_group)
print('Процент просрочек по кредитам клиентов никогда не состоявший в офицальном браке:',
      family_group.loc[[1, 4]]['debt'].sum() * 100 / family_group.loc[[1, 4]]['family_status_id'].sum())
print('Процент просрочек по кредитам клиентов состоящих (состоявших) в офицальном браке:',
      family_group.loc[[0, 2, 3]]['debt'].sum() * 100 / family_group.loc[[0, 2, 3]]['family_status_id'].sum())

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4
                  family_status_id  debt  persent_debt
family_status_id                                      
0                            12339   931      7.545182
1                             4151   388      9.347145
2                              959    63      6.569343
3                             1195    85      7.112971
4                             2810   274      9.750890
Процент просрочек по кредитам клиентов никогда не состоявший в офицальном браке: 9.51012785519322
Процент просрочек по кредитам клиентов состоящих (состоявших) в офицальном браке: 7.4449734354515975


##### <a id='toc1_1_9_1_4_'></a>[Вывод 2:](#toc0_)

Такая зависемость имеется - клиенты, состоящие либо состоявшие в браке реже всего допускают образование задолженности (7,4%), из них овдовевшие реже всех допускают просрочки (6.5%), а чаще всего женатые / замужем (7.5%). Разведенные клиенты просрачивали выплаты в 7.1% случаев.

Клиенты, не состоящие в браке, в том числе состоящие в "гражданском браке", допускают просрочки по кредитам чаще (9,5%). 

##### <a id='toc1_1_9_1_5_'></a>[Вопрос 3:](#toc0_)

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [55]:
data_income_category = data.groupby(['total_income_category']).agg({'total_income_category':'count', 'debt':'sum'})
data_income_category['persent_debt'] = data_income_category['debt'] * 100 / data_income_category['total_income_category']
data_income_category

,total_income_category,debt,persent_debt
total_income_category,,,
A,25,2,8.000000
B,5041,356,7.062091
C,16016,1360,8.491508
D,350,21,6.000000
E,22,2,9.090909


##### <a id='toc1_1_9_1_6_'></a>[Вывод 3:](#toc0_)

    Исходя из данных представленных в датасете размер доходов и факт наличия просрочки по кредиту не являются величинами обратно или прямо пропорциональными. 
    Наименьший объем просрочек наблюдается в категории D (6%), что меньше, чем в категориях с более высоким уровнем дохода - B(7%), C(8%). 
    Делать выводы о поведении категорий A, E не представляется возможным в связи с малой выборкой - 25 и 22 кредита соответственно.

##### <a id='toc1_1_9_1_7_'></a>[Вопрос 4:](#toc0_)

Как разные цели кредита влияют на его возврат в срок?

In [56]:
data_income_category = data.groupby(['purpose_category']).agg({'purpose_category':'count', 'debt':'sum'})
data_income_category['percent_purpose_category'] = data_income_category['debt'] * 100 / data_income_category['purpose_category']
print(data_income_category)

                          purpose_category  debt  percent_purpose_category
purpose_category                                                          
операции с автомобилем                4306   403                  9.359034
операции с недвижимостью             10811   782                  7.233373
получение образования                 4013   370                  9.220035
проведение свадьбы                    2324   186                  8.003442


##### <a id='toc1_1_9_1_8_'></a>[Вывод 4:](#toc0_)

Наименьший процент просрочек по кредитам имеется, полученных для: операций с недвижимостью (7,2%) и проведения свадьбы (8%). 
Наибольший процент просрочек по кредитам имеется, полученных для: операций с автомобилем (9,3%) и получения образования (9,2%). 

## <a id='toc1_2_'></a>[Общий вывод:](#toc0_)

Семейное положение и факт наличия детей вляет на факт погашения кредита в срок. 

Так, у людей с детьми чаще просрочки по кредитам (9.2%), чем у бездетных (7.5%). 
Клиенты, состоящие либо состоявшие в браке реже всего допускают образование задолженности (7,4%), из них овдовевшие реже всех допускают просрочку (6.5%), а чаще всего женатые / замужем (7.5%). Разведенные клиенты просрачивали выплаты в 7.1% случаев. клиенты, не состоящие в браке, в том числе состоящие в "гражданском браке", допускают просрочки по кредитам чаще (9,5%).

Однако на факт погашения кредита в срок также влияют: цель кредита, уровень дохода клиента.

Так, наименьший процент просрочек по кредитам имеется, полученных для: операций с недвижимостью (7,2%) и проведения свадьбы (8%). Наибольший процент просрочек по кредитам имеется, полученных для: операций с автомобилем (9,3%) и получения образования (9,2%). По уровню дохода наименьший объем просрочек наблюдается в категории D (6%), что меньше, чем в категориях с более высоким уровнем дохода - B(7%), C(8%).